## USING UNSLOTH TO FASTLY FINETUNE OUR MODELL


In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# Install Flash Attention 2 for softcapping support
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

## loading the pre-quantized unsloth models


In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
#     "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
#     "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
#     "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
#     "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
#     "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
#     "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
#     "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
#     "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!d
#     "unsloth/Phi-3-medium-4k-instruct",
#     "unsloth/gemma-2-9b-bnb-4bit",
#     "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
#     "unsloth/gemma-2-2b-bnb-4bit",             # New small Gemma model!
# ] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/zephyr-sft-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.11.7: Fast Mistral patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/511 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.11.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Loading the dataset

In [4]:
import json

# Open and read the JSON file
with open('/content/sarcasm_llaava.json', 'r') as file:
    data = json.load(file)

# Print the data
# print(data)

In [5]:
import pandas as pd
df = pd.DataFrame(data.items(), columns=['Filename', 'Description'])


# Display the DataFrame
print(df.head())

                 Filename                                        Description
0  919397113967951872.jpg  \n\n\nThe image you've provided appears to be ...
1  706531433272094720.jpg  \n\n\nThe image shows a Lego Star Wars Millenn...
2  833053711001870337.jpg  \n\n\nThe image depicts two cars parked in a p...
3  871222723078373377.jpg  \n\n\nThe image shows a cup of food, presumabl...
4  932353655130480640.jpg  \n\n\nThe image shows a pair of Christian Loub...


In [6]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


## Merging the common sense reasoning with more dataset

In [7]:
import pandas as pd

# File paths
path_to_train = '/content/train_df.tsv'
path_to_val = '/content/val_df.tsv'
path_to_test = '/content/test_df.tsv'

# Column names
columns = ["ID", "caption", "Explanation"]

# Load train, validation, and test files
train_df = pd.read_csv(path_to_train, sep='\t', header=None, names=columns)
val_df = pd.read_csv(path_to_val, sep='\t', header=None, names=columns)
test_df = pd.read_csv(path_to_test, sep='\t', header=None, names=columns)

# Assume 'df' is the DataFrame to merge with
# Clean 'df' to match IDs
df['Filename'] = df['Filename'].str.replace('.jpg', '', regex=False)
df.rename(columns={'Filename': 'ID'}, inplace=True)

# Step 3: Merge train, validation, and test DataFrames with df
train_combined = pd.merge(train_df, df, on='ID', how='inner')
val_combined = pd.merge(val_df, df, on='ID', how='inner')
test_combined = pd.merge(test_df, df, on='ID', how='inner')

# Optionally, combine all the merged DataFrames
all_combined = pd.concat([train_combined, val_combined, test_combined], ignore_index=True)

# Display the combined DataFrame
# print("Train Combined:")
# print(train_combined.head())
# print("\nValidation Combined:")
# print(val_combined.head())
# print("\nTest Combined:")
# print(test_combined.head())
# print("\nAll Combined:")
# print(all_combined.head())


In [8]:
def format_dataset(df):
    formatted_data = []
    for _, row in df.iterrows():
        formatted_data.append({
            "instruction": "Provide a sarcasm explanation.",
            "input": f"Caption: {row['caption']}\nDescription: {row['Description']}",
            "output": row['Explanation']
        })
    return pd.DataFrame(formatted_data)

# Format train, validation, and test datasets
train_formatted = format_dataset(train_combined)
val_formatted = format_dataset(val_combined)
test_formatted = format_dataset(test_combined)

# Save formatted datasets (optional)
train_formatted.to_json("train_formatted.json", orient="records", lines=True)
val_formatted.to_json("val_formatted.json", orient="records", lines=True)
test_formatted.to_json("test_formatted.json", orient="records", lines=True)

# Print an example from the formatted training dataset
(train_formatted.head())

,instruction,input,output
0,Provide a sarcasm explanation.,Caption: 'check out the pulse pounding excitem...,the people are bored of waiting in line <num> ...
1,Provide a sarcasm explanation.,Caption: 'proud to have a <user> who attacks p...,it's sad to have a <user> who attacks private ...
2,Provide a sarcasm explanation.,Caption: 'the view from my house ........ yay ...,the author has a terrible view of everything c...
3,Provide a sarcasm explanation.,Caption: 'these # colombo # jaffna # vavuniya ...,these # colombo # jaffna # vavuniya luxury bus...
4,Provide a sarcasm explanation.,"Caption: Thanks North Carolina, I was worried ...",the author was worried that they were gonna hi...


In [9]:
train_combined.head()

,ID,caption,Explanation,Description
0,931874353976938497,'check out the pulse pounding excitement of # ...,the people are bored of waiting in line <num> ...,\n\n\nThe image depicts a group of people sitt...
1,880425829246922752,'proud to have a <user> who attacks private ci...,it's sad to have a <user> who attacks private ...,\n\n\nThe image appears to be a humorous take ...
2,690915881082343424,'the view from my house ........ yay # pasnow...,the author has a terrible view of everything c...,\n\n\nThe image depicts a snow-covered landsca...
3,915228456757059585,'these # colombo # jaffna # vavuniya luxury bu...,these # colombo # jaffna # vavuniya luxury bus...,\n\n\n# -*-
4,494194068998468686_25639236,"Thanks North Carolina, I was worried we weren'...",the author was worried that they were gonna hi...,\n\n\nThe image depicts a highway scene with m...


<a name="Data"></a>
### Data Prep
# Creating the prompt format for finetuning

In [10]:
import pandas as pd
from datasets import Dataset

# Define the new prompt format
sarcasm_prompt = """Below is a task that involves providing a sarcasm explanation based on the given caption and description.

### Task:
{}

### Caption:
{}

### Description:
{}

### Sarcasm Explanation:
{}"""

EOS_TOKEN = "<|endoftext|>"  # Replace with your tokenizer's EOS token

# Function to format the dataset
def formatting_prompts_func(df):
    tasks = ["Provide a sarcasm explanation."] * len(df)  # Task remains constant
    captions = df["caption"].tolist()
    descriptions = df["Description"].tolist()
    explanations = df["Explanation"].tolist()
    texts = []
    for task, caption, description, explanation in zip(tasks, captions, descriptions, explanations):
        # Format the data into the desired prompt format
        text = sarcasm_prompt.format(task, caption, description, explanation) + EOS_TOKEN
        texts.append({"text": text})
    return texts

# Process train, validation, and test DataFrames
train_texts = formatting_prompts_func(train_combined)
val_texts = formatting_prompts_func(val_combined)
test_texts = formatting_prompts_func(test_combined)

# Convert to Hugging Face Dataset format
dataset = Dataset.from_list(train_texts)
val_dataset = Dataset.from_list(val_texts)
test_dataset = Dataset.from_list(test_texts)

# Save formatted datasets (optional)
dataset.save_to_disk("formatted_train_dataset")
val_dataset.save_to_disk("formatted_val_dataset")
test_dataset.save_to_disk("formatted_test_dataset")

# Print an example from the training dataset
print(dataset[0])


Saving the dataset (0/1 shards):   0%|          | 0/2983 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/175 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/352 [00:00<?, ? examples/s]

{'text': "Below is a task that involves providing a sarcasm explanation based on the given caption and description.\n\n### Task:\nProvide a sarcasm explanation.\n\n### Caption:\n'check out the pulse pounding excitement of # paxunplugged of waiting in line <num> hours before game sign up begins . waiting in line , what a gaming convention is all about . '\n\n### Description:\n\n\n\nThe image depicts a group of people sitting on the floor in what appears to be a convention or conference hall, with some standing and others sitting on the floor. The setting suggests a social gathering or event, possibly related to a conference or convention. The individuals are casually dressed, and some are engaged in conversation or looking at their phones.\n\n1. Implied Meaning:\nThe sarcastic or humorous message being conveyed here could be related to the contrast between the expectations of a formal event and the actual experience of attendees. The image might be commenting on the reality that sometim

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer` for better control

In [11]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Create the output directory path in Google Drive
output_dir = "/content/drive/MyDrive/1a-h3wtFt0zQQ66H5rinw2rEwzX72-nuZ"
os.makedirs(output_dir, exist_ok=True)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 2,
        max_steps = 100,  # Set to -1 to use num_train_epochs instead
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = output_dir,
        report_to = "none",
        save_strategy = "epoch",  # Save checkpoint every epoch
    ),
)

Mounted at /content/drive


Map (num_proc=2):   0%|          | 0/2983 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [12]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")

print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
4.625 GB of memory reserved.


In [13]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,983 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.189500
2,2.202400
3,2.045300
4,1.869900
5,1.851100
6,1.704300
7,1.451400
8,1.474400
9,1.294600
10,1.189400


In [14]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

892.7458 seconds used for training.
14.88 minutes used for training.
Peak reserved memory = 5.457 GB.
Peak reserved memory for training = 0.832 GB.
Peak reserved memory % of max memory = 37.002 %.
Peak reserved memory for training % of max memory = 5.641 %.


In [15]:
test_combined.head()

,ID,caption,Explanation,Description
0,685491413409112065,'nothing better than # design of convention ce...,the author hates the design of this convention...,\n\n\nThe image depicts a hallway with a check...
1,700183392969756672,'oh i so love working late from home # work #...,the author hates working late from home.,"\n\n\nIn the image, a person is taking a selfi..."
2,928753954745475072,'yeaah ! buddy o miracle worker # infj emoji_...,"your anxiety is not cured when someone says ""d...",\n\n\n1. Implied Meaning:\nThe image shows a p...
3,935133439011049473,'rt <user> : something different ..... a delay...,the author is pissed to watch a full train lea...,\n\n\nThe image depicts a man sitting at a des...
4,933466049697198080,'oh really linkedin ? thanks for the super use...,the author doesn't find such notifications fro...,\n\n\nThe image appears to be a screenshot of ...


In [16]:
# test_combined.iloc[0]["caption"]
sample_test=test_combined.iloc[0]
sample_caption=sample_test["caption"]
sample_description=sample_test["Description"]
sample_explanation=sample_test["Explanation"]
# sample_explanation

<a name="Inference"></a>
### Inference
## Example inference

In [17]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
sample_test=test_combined.iloc[0]
sample_caption=sample_test["caption"]
sample_description=sample_test["Description"]
sample_explanation=sample_test["Explanation"]

inputs = tokenizer(
[
    f"""Below is a task that involves providing a sarcasm explanation based on the given caption and description.

### Task:
{"Provide a sarcasm explanation"}

### Caption:
{sample_caption}

### Description:
{sample_description}

### Sarcasm Explanation:
"""
], return_tensors = "pt").to("cuda")
input_text=tokenizer.batch_decode(inputs["input_ids"])[0]
# print(input_text)
outputs = model.generate(**inputs, max_new_tokens = 40, use_cache = True)

print(tokenizer.batch_decode(outputs)[0][len(input_text):])

the author hates the design of convention centers, it's bad design.<|endoftext|></s>


In [18]:
import pandas as pd
from tqdm import tqdm

# Assuming 'test_combined' is your DataFrame that contains 'caption', 'Description', and 'Explanation'
# Define a list to hold all the data
dataset = []

# Iterate through your DataFrame (test_combined)
for index, row in tqdm(test_combined.iterrows(), total=test_combined.shape[0]):
    sample_caption = row["caption"]
    sample_description = row["Description"]
    sample_explanation = row["Explanation"]

    # Tokenizing the input text
    inputs = tokenizer(
        [
            f"""Below is a task that involves providing a sarcasm explanation based on the given caption and description.

### Task:
Provide a sarcasm explanation

### Caption:
{sample_caption}

### Description:
{sample_description}

### Sarcasm Explanation:
"""
        ], return_tensors="pt").to("cuda")

    # Generate the sarcasm explanation
    outputs = model.generate(**inputs, max_new_tokens=40, use_cache=True)
    sarcasm_explanation = tokenizer.batch_decode(outputs)[0][len(tokenizer.batch_decode(inputs["input_ids"])[0]):]

    # Append the data to the dataset list
    dataset.append({
        "caption": sample_caption,
        "description": sample_description,
        "sarcasm_explanation": sarcasm_explanation,
        "original_explanation": sample_explanation
    })

# Convert the dataset into a pandas DataFrame
df = pd.DataFrame(dataset)

# Save the DataFrame as a CSV file
df.to_csv("sarcasm_explanation_dataset.csv", index=False)

# Optionally, if you want to save as JSON
df.to_json("sarcasm_explanation_dataset.json", orient="records", lines=True)

print("Dataset has been saved successfully.")


100%|██████████| 352/352 [14:07<00:00,  2.41s/it]

Dataset has been saved successfully.


In [19]:
import re

def clean_text(text):
    # Remove leading/trailing whitespace
    text = text.strip()

    # Remove unwanted quotation marks
    text = text.replace('"', '')  # Remove extra quotes

    # Remove hashtags (e.g., #hashtag)
    text = re.sub(r'#\S+', '', text)  # Remove any words that start with a hash

    # Remove any special tokens like <|endoftext|> or <|startoftext|> and similar
    text = re.sub(r'<\|.*?\|>', '', text)  # Match any text between <| and |>

    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)

    # Fix punctuation: ensure one space after punctuation marks
    text = re.sub(r'([.,;!?])([^\s])', r'\1 \2', text)  # Add space after punctuation if missing
    text = re.sub(r'\s([.,;!?])', r'\1', text)  # Remove space before punctuation if present

    # Capitalize the first letter of the text
    text = text[0].upper() + text[1:] if text else text

    return text

# Apply the cleaning function to the 'sarcasm_explanation' column
df['sarcasm_explanation'] = df['sarcasm_explanation'].apply(clean_text)

# Optionally, check the cleaned dataframe
print(df[['sarcasm_explanation']].head())

# Save the cleaned DataFrame as CSV or JSON
df.to_csv("cleaned_sarcasm_explanation_dataset.csv", index=False)
df.to_json("cleaned_sarcasm_explanation_dataset.json", orient="records", lines=True)

print("DataFrame with cleaned sarcasm explanations has been saved successfully.")


                                 sarcasm_explanation
0  The author hates the design of convention cent...
1  The author hates working late from home. Sarca...
2  The author is pissed at buddy o miracle worker...
3  The author is pissed at <user> for having to w...
4  The author doesn't want to follow bill gates, ...
DataFrame with cleaned sarcasm explanations has been saved successfully.


In [21]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.9 MB/s eta 0:00:00


In [22]:
!pip install rouge-score
!pip install bert-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=21b8464dcab190c8a3d45d964f106c9e60b320107cbf6d3b1d4012b87ea44ca4
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.0 MB/s eta 0:00:00


In [23]:
import pandas as pd
import evaluate
from rouge_score import rouge_scorer
import nltk
from bert_score import score

# Load the dataset
data = df

# Define the metrics
rouge = evaluate.load('rouge')
meteor = evaluate.load('meteor')
bleu = evaluate.load('bleu')

# Preprocessing the columns
captions = data['caption'].tolist()
descriptions = data['description'].tolist()
sarcasm_explanation = data['sarcasm_explanation'].tolist()
original_explanation = data['original_explanation'].tolist()

# A helper function to calculate ROUGE scores
def compute_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    results = {'rouge1': [], 'rouge2': [], 'rougeL': []}
    for pred, ref in zip(predictions, references):
        score = scorer.score(ref, pred)
        for key in results:
            results[key].append(score[key].fmeasure)
    return results

# A helper function to compute BERTScore
def compute_bert_score(predictions, references):
    P, R, F1 = score(predictions, references, lang='en')
    return P.mean().item(), R.mean().item(), F1.mean().item()

# Evaluate each column
def evaluate_column(predictions, references):
    # ROUGE Scores
    rouge_results = compute_rouge(predictions, references)

    # BERTScore
    bert_precision, bert_recall, bert_f1 = compute_bert_score(predictions, references)

    # METEOR Score
    meteor_score_value = meteor.compute(predictions=predictions, references=references)['meteor']

    # BLEU Score
    bleu_score_value = bleu.compute(predictions=predictions, references=references)['bleu']

    return {
        'rouge1': sum(rouge_results['rouge1']) / len(rouge_results['rouge1']),
        'rouge2': sum(rouge_results['rouge2']) / len(rouge_results['rouge2']),
        'rougeL': sum(rouge_results['rougeL']) / len(rouge_results['rougeL']),
        'bert_precision': bert_precision,
        'bert_recall': bert_recall,
        'bert_f1': bert_f1,
        'meteor_score': meteor_score_value,
        'bleu_score': bleu_score_value
    }

# Example: Evaluate using the 'sarcasm_explanation' column as predictions, and 'original_explanation' as references
sarcasm_results = evaluate_column(sarcasm_explanation, original_explanation)
print("Sarcasm Explanation Evaluation:", sarcasm_results)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Sarcasm Explanation Evaluation: {'rouge1': 0.4927262002452137, 'rouge2': 0.3142046525376663, 'rougeL': 0.4477525062839151, 'bert_precision': 0.8903558254241943, 'bert_recall': 0.9091058373451233, 'bert_f1': 0.899451494216919, 'meteor_score': 0.5083557413694376, 'bleu_score': 0.221284488582767}


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model_2") # Local saving
tokenizer.save_pretrained("lora_model_2")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

In [ ]:
import torch

# Assuming `model` and `tokenizer` are already defined
model_path = "lora_model_2"

# Save both model and tokenizer to a dictionary
model_and_tokenizer = {
    'model': model.state_dict(),
    'tokenizer': tokenizer
}

# Save to a single .pt file
torch.save(model_and_tokenizer, f"{model_path}.pt")

print(f"Model and tokenizer saved to {model_path}.pt")


In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the saved dictionary
model_and_tokenizer = torch.load("lora_model_2.pt")

# Reconstruct the model
# Specify the model architecture (you may change this to the appropriate model type for your use case)
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
model.load_state_dict(model_and_tokenizer['model'])  # Load model weights

# Load the tokenizer
tokenizer = model_and_tokenizer['tokenizer']

print("Model and tokenizer loaded successfully.")


Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.